In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import win32com.client
import sqlite3

import sys
from enum import Enum
import time
import pythoncom

# matplotlib 그래프 한글지원
import platform
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus']= False
if platform.system()== 'Windows':
    path= 'c:/Windows/Fonts/malgun.ttf'
    font_name= font_manager.FontProperties(fname=path).get_name()
    rc('font', family= font_name)
elif platform.system()=='Darwin':
    rc('font', family= 'AppleGothic')
else:
    print('포트를 지원하지 않습니다.')

In [2]:
instCpCybos= win32com.client.Dispatch("CpUtil.CpCybos")

# 연결 여부 체크
bConnect = instCpCybos.IsConnect
if (bConnect == 1):
    print("연결되었음.")
else:
    print("연결되지않음.")
    

연결되었음.


In [3]:
#각종 코드정보 및 코드리스트를 얻는다.
instCpCodeMgr= win32com.client.Dispatch('CpUtil.CpCodeMgr')

codeList= instCpCodeMgr.GetStockListByMarket(1) # 코스피 종목코드를 튜플형태로 반환
kospi_list= []
for code in codeList:
    name= instCpCodeMgr.CodeToName(code)
    gubun= instCpCodeMgr.GetStockSectionKind(code) # 주권 1 ETF 10
    gubun1= instCpCodeMgr.GetStockMarketKind(code) # 코스피 1 코스닥 2
    업종= instCpCodeMgr.GetStockIndustryCode(code) # 업종구분
    업종명= instCpCodeMgr.GetIndustryName(업종) # 업종명 구분
    코스피200= instCpCodeMgr.GetStockKospi200Kind(code) # KOSPI200 여부
    관리= instCpCodeMgr.GetStockStatusKind(code) # 관리구분: 정상0, 거래정지1, 거래중단2
    상장일= instCpCodeMgr.GetStockListedDate(code)   
    kospi_list.append([code, name, gubun, gubun1, 업종, 업종명, 코스피200, 관리, 상장일])   

kospi_df= pd.DataFrame(kospi_list, columns=['종목코드','종목명','구분','소속','업종','업종명','KOSPI200','상태','상장일'])
kospi_df= kospi_df[kospi_df['구분']== 1] # ETF제외 
print(kospi_df.shape)


codeList= instCpCodeMgr.GetStockListByMarket(2) # 코스닥 종목코드를 튜플형태로 반환
kosdaq_list= []
for code in codeList:
    name= instCpCodeMgr.CodeToName(code)
    gubun= instCpCodeMgr.GetStockSectionKind(code) # 주권 1 ETF 10
    gubun1= instCpCodeMgr.GetStockMarketKind(code) # 코스피 1 코스닥 2
    업종= instCpCodeMgr.GetStockIndustryCode(code) # 업종 구분
    업종명= instCpCodeMgr.GetIndustryName(업종) # 업종명 구분
    코스피200= instCpCodeMgr.GetStockKospi200Kind(code) # KOSPI200 여부
    관리= instCpCodeMgr.GetStockStatusKind(code) # 관리구분: 정상0, 거래정지1, 거래중단2
    상장일= instCpCodeMgr.GetStockListedDate(code)   
    kosdaq_list.append([code, name, gubun, gubun1, 업종, 업종명, 코스피200, 관리, 상장일])   

kosdaq_df= pd.DataFrame(kosdaq_list, columns=['종목코드','종목명','구분','소속','업종','업종명','KOSPI200','상태','상장일'])
kosdaq_df= kosdaq_df[kosdaq_df['구분']== 1] # ETF제외 
print(kosdaq_df.shape)

(884, 9)
(1315, 9)


In [3]:
# 종목코드 데이터 DB불러오기 
kospi_df= None
kosdaq_df= None
with sqlite3.connect('./data/StockData3.sqlite') as conn:
    kospi_df= pd.read_sql("SELECT * From '코스피_종목코드'", con=conn) # if_exists='append'
    kosdaq_df= pd.read_sql("SELECT * From '코스닥_종목코드'", con=conn) # if_exists='append'    
print(kospi_df.shape)
print(kosdaq_df.shape)

(884, 9)
(1315, 9)


In [4]:
# kospi_df= pd.concat([kospi_df, kosdaq_df])
# print(kospi_df.shape)

(2190, 9)


In [4]:
# 과거데이터 구하기
# 주식, 업종, ELW의 차트데이터를 수신합니다.
def getStockChart(종목코드='', 요청구분='', 종료일='', 시작일='', 요청건수=''):
    
    inst.SetInputValue(0, 종목코드) # 0 종목코드
    inst.SetInputValue(1, ord(요청구분)) # 1 요청구분 1기간으로 요청 2 개수로 요청 #ord 문자열을 아스키값으로 변환
    if 요청구분 == '1':
        inst.SetInputValue(2, 종료일)# 2 요청종료일
        inst.SetInputValue(3, 시작일)# 3 요청시작일
    elif 요청구분 == '2':
        inst.SetInputValue(4, int(요청건수)) # 4 요청할 데이터의 개수
    else:
        return print('요청구분설정오류: 1기간 2개수')
    inst.SetInputValue(5,(0,2,3,4,5,8,9,12,13)) # 5 필드 또는 필드배열 (0,1,2,3~37, 5는 종가)
    inst.SetInputValue(6, ord('D')) # 6 차트구분 D 일
    inst.SetInputValue(9, ord('1')) # 9 수정주가 반영 유무
    
    연속조회 = True
    result=[]

    while 연속조회 == True :        
     
        # 제한을 하기 전까지의 남은 요청개수, 15초에 최대60건
        remainCount = instCpCybos.GetLimitRemainCount(1)  # 1 시세관련요청
        if remainCount <= 0: 
            print('시세 연속 조회 제한 대기:',int(instCpCybos.LimitRequestRemainTime / 1000 + 5), '초')
            time.sleep(instCpCybos.LimitRequestRemainTime / 1000 + 5)

        inst.BlockRequest()
        
        #현재가 통신 및 통신 에러 처리
        rqStatus = inst.GetDibStatus()
        #print('통신상태', rqStatus, objRq.GetDibMsg1())
        if rqStatus != 0:
            print('통신실패')
            break 

        numData= inst.GetHeaderValue(3) # 3 수신개수
        numField= inst.GetHeaderValue(1) # 1 필드개수
        print('행:',numData,'열:',numField, end=' ')

        for i in range(numData):
            날짜= str(inst.GetDataValue(0,i))
            시가= int(inst.GetDataValue(1,i))
            고가= int(inst.GetDataValue(2,i))
            저가= int(inst.GetDataValue(3,i))
            종가= int(inst.GetDataValue(4,i))
            거래량= float(inst.GetDataValue(5,i))
            거래대금= float(inst.GetDataValue(6,i))
            상장주식수= float(inst.GetDataValue(7,i))
            시가총액= float(inst.GetDataValue(8,i))
            # 종목코드
            result.append([종목코드,날짜,시가,고가,저가,종가,거래량,거래대금,상장주식수,시가총액])         

        if inst.Continue != True:
            연속조회 = False
            break
            
       
    df_local= pd.DataFrame(result, columns=['종목코드','날짜','시가','고가','저가','종가',
                                      '거래량','거래대금','상장주식수','시가총액'])   
    print(종목코드,'조회완료',df_local.shape)
            
    return df_local


In [5]:
inst= win32com.client.Dispatch('CpSysDib.StockChart')

In [6]:
# 부분 데이터 업데이트

df= None
cnt= 0
for code in kosdaq_df.종목코드:
    df0 = getStockChart(종목코드= code, 요청구분='2', 요청건수= '20')
    time.sleep(0.1)
    df= pd.concat([df, df0])
    cnt= cnt+1
    print(cnt)
    
print(df.shape)
print(df.shape[0] / 2)

행: 20 열: 9 A000250 조회완료 (20, 10)
1
행: 20 열: 9 A000440 조회완료 (20, 10)
2
행: 20 열: 9 A001000 조회완료 (20, 10)
3
행: 20 열: 9 A001540 조회완료 (20, 10)
4
행: 20 열: 9 A001810 조회완료 (20, 10)
5
행: 20 열: 9 A001840 조회완료 (20, 10)
6
행: 20 열: 9 A002230 조회완료 (20, 10)
7
행: 20 열: 9 A002290 조회완료 (20, 10)
8
행: 20 열: 9 A002680 조회완료 (20, 10)
9
행: 20 열: 9 A002800 조회완료 (20, 10)
10
행: 20 열: 9 A003100 조회완료 (20, 10)
11
행: 20 열: 9 A003310 조회완료 (20, 10)
12
행: 20 열: 9 A003380 조회완료 (20, 10)
13
행: 20 열: 9 A003670 조회완료 (20, 10)
14
행: 20 열: 9 A003800 조회완료 (20, 10)
15
행: 20 열: 9 A004590 조회완료 (20, 10)
16
행: 20 열: 9 A004650 조회완료 (20, 10)
17
행: 20 열: 9 A004780 조회완료 (20, 10)
18
행: 20 열: 9 A005160 조회완료 (20, 10)
19
행: 20 열: 9 A005290 조회완료 (20, 10)
20
행: 20 열: 9 A005670 조회완료 (20, 10)
21
행: 20 열: 9 A005710 조회완료 (20, 10)
22
행: 20 열: 9 A005860 조회완료 (20, 10)
23
행: 20 열: 9 A005990 조회완료 (20, 10)
24
행: 20 열: 9 A006050 조회완료 (20, 10)
25
행: 20 열: 9 A006140 조회완료 (20, 10)
26
행: 20 열: 9 A006580 조회완료 (20, 10)
27
행: 20 열: 9 A006620 조회완료 (20, 10)
28
행

224
행: 20 열: 9 A035460 조회완료 (20, 10)
225
행: 20 열: 9 A035480 조회완료 (20, 10)
226
행: 20 열: 9 A035600 조회완료 (20, 10)
227
행: 20 열: 9 A035610 조회완료 (20, 10)
228
행: 20 열: 9 A035620 조회완료 (20, 10)
229
행: 20 열: 9 A035760 조회완료 (20, 10)
230
행: 20 열: 9 A035810 조회완료 (20, 10)
231
행: 20 열: 9 A035890 조회완료 (20, 10)
232
행: 20 열: 9 A035900 조회완료 (20, 10)
233
행: 20 열: 9 A036000 조회완료 (20, 10)
234
행: 20 열: 9 A036010 조회완료 (20, 10)
235
행: 20 열: 9 A036030 조회완료 (20, 10)
236
행: 20 열: 9 A036090 조회완료 (20, 10)
237
행: 20 열: 9 A036120 조회완료 (20, 10)
238
행: 20 열: 9 A036170 조회완료 (20, 10)
239
행: 20 열: 9 A036180 조회완료 (20, 10)
240
시세 연속 조회 제한 대기: 12 초
행: 20 열: 9 A036190 조회완료 (20, 10)
241
행: 20 열: 9 A036200 조회완료 (20, 10)
242
행: 20 열: 9 A036260 조회완료 (20, 10)
243
행: 20 열: 9 A036420 조회완료 (20, 10)
244
행: 20 열: 9 A036480 조회완료 (20, 10)
245
행: 20 열: 9 A036490 조회완료 (20, 10)
246
행: 20 열: 9 A036540 조회완료 (20, 10)
247
행: 20 열: 9 A036560 조회완료 (20, 10)
248
행: 20 열: 9 A036620 조회완료 (20, 10)
249
행: 20 열: 9 A036630 조회완료 (20, 10)
250
행: 20 열: 9 A0

444
행: 20 열: 9 A053060 조회완료 (20, 10)
445
행: 20 열: 9 A053110 조회완료 (20, 10)
446
행: 20 열: 9 A053160 조회완료 (20, 10)
447
행: 20 열: 9 A053260 조회완료 (20, 10)
448
행: 20 열: 9 A053270 조회완료 (20, 10)
449
행: 20 열: 9 A053280 조회완료 (20, 10)
450
행: 20 열: 9 A053290 조회완료 (20, 10)
451
행: 20 열: 9 A053300 조회완료 (20, 10)
452
행: 20 열: 9 A053350 조회완료 (20, 10)
453
행: 20 열: 9 A053450 조회완료 (20, 10)
454
행: 20 열: 9 A053580 조회완료 (20, 10)
455
행: 20 열: 9 A053590 조회완료 (20, 10)
456
행: 20 열: 9 A053610 조회완료 (20, 10)
457
행: 20 열: 9 A053620 조회완료 (20, 10)
458
행: 20 열: 9 A053660 조회완료 (20, 10)
459
행: 20 열: 9 A053700 조회완료 (20, 10)
460
행: 20 열: 9 A053800 조회완료 (20, 10)
461
행: 20 열: 9 A053950 조회완료 (20, 10)
462
행: 20 열: 9 A053980 조회완료 (20, 10)
463
행: 20 열: 9 A054040 조회완료 (20, 10)
464
행: 20 열: 9 A054050 조회완료 (20, 10)
465
행: 20 열: 9 A054090 조회완료 (20, 10)
466
행: 20 열: 9 A054180 조회완료 (20, 10)
467
행: 20 열: 9 A054210 조회완료 (20, 10)
468
행: 20 열: 9 A054220 조회완료 (20, 10)
469
행: 20 열: 9 A054300 조회완료 (20, 10)
470
행: 20 열: 9 A054340 조회완료 (20, 10)
4

664
행: 20 열: 9 A079650 조회완료 (20, 10)
665
행: 20 열: 9 A079810 조회완료 (20, 10)
666
행: 20 열: 9 A079940 조회완료 (20, 10)
667
행: 20 열: 9 A079950 조회완료 (20, 10)
668
행: 20 열: 9 A079960 조회완료 (20, 10)
669
행: 20 열: 9 A079970 조회완료 (20, 10)
670
행: 20 열: 9 A080000 조회완료 (20, 10)
671
행: 20 열: 9 A080010 조회완료 (20, 10)
672
행: 20 열: 9 A080160 조회완료 (20, 10)
673
행: 20 열: 9 A080220 조회완료 (20, 10)
674
행: 20 열: 9 A080420 조회완료 (20, 10)
675
행: 20 열: 9 A080440 조회완료 (20, 10)
676
행: 20 열: 9 A080470 조회완료 (20, 10)
677
행: 20 열: 9 A080520 조회완료 (20, 10)
678
행: 20 열: 9 A080530 조회완료 (20, 10)
679
행: 20 열: 9 A080580 조회완료 (20, 10)
680
행: 20 열: 9 A080720 조회완료 (20, 10)
681
행: 20 열: 9 A081150 조회완료 (20, 10)
682
행: 20 열: 9 A081580 조회완료 (20, 10)
683
행: 20 열: 9 A082210 조회완료 (20, 10)
684
행: 20 열: 9 A082270 조회완료 (20, 10)
685
행: 20 열: 9 A082660 조회완료 (20, 10)
686
행: 20 열: 9 A082800 조회완료 (20, 10)
687
행: 20 열: 9 A082850 조회완료 (20, 10)
688
행: 20 열: 9 A082920 조회완료 (20, 10)
689
행: 20 열: 9 A083310 조회완료 (20, 10)
690
행: 20 열: 9 A083450 조회완료 (20, 10)
6

884
행: 20 열: 9 A115440 조회완료 (20, 10)
885
행: 20 열: 9 A115450 조회완료 (20, 10)
886
행: 20 열: 9 A115480 조회완료 (20, 10)
887
행: 20 열: 9 A115500 조회완료 (20, 10)
888
행: 20 열: 9 A115530 조회완료 (20, 10)
889
행: 20 열: 9 A115570 조회완료 (20, 10)
890
행: 20 열: 9 A115610 조회완료 (20, 10)
891
행: 20 열: 9 A115960 조회완료 (20, 10)
892
행: 20 열: 9 A117670 조회완료 (20, 10)
893
행: 20 열: 9 A117730 조회완료 (20, 10)
894
행: 20 열: 9 A118990 조회완료 (20, 10)
895
행: 20 열: 9 A119500 조회완료 (20, 10)
896
행: 20 열: 9 A119610 조회완료 (20, 10)
897
행: 20 열: 9 A119830 조회완료 (20, 10)
898
행: 20 열: 9 A119850 조회완료 (20, 10)
899
행: 20 열: 9 A119860 조회완료 (20, 10)
900
시세 연속 조회 제한 대기: 12 초
행: 20 열: 9 A120240 조회완료 (20, 10)
901
행: 20 열: 9 A121440 조회완료 (20, 10)
902
행: 20 열: 9 A121600 조회완료 (20, 10)
903
행: 20 열: 9 A121800 조회완료 (20, 10)
904
행: 20 열: 9 A121850 조회완료 (20, 10)
905
행: 20 열: 9 A121890 조회완료 (20, 10)
906
행: 20 열: 9 A122310 조회완료 (20, 10)
907
행: 20 열: 9 A122350 조회완료 (20, 10)
908
행: 20 열: 9 A122450 조회완료 (20, 10)
909
행: 20 열: 9 A122640 조회완료 (20, 10)
910
행: 20 열: 9 A1

1102
행: 20 열: 9 A208710 조회완료 (20, 10)
1103
행: 20 열: 9 A208860 조회완료 (20, 10)
1104
행: 20 열: 9 A211270 조회완료 (20, 10)
1105
행: 20 열: 9 A212560 조회완료 (20, 10)
1106
행: 20 열: 9 A213090 조회완료 (20, 10)
1107
행: 20 열: 9 A213420 조회완료 (20, 10)
1108
행: 20 열: 9 A214150 조회완료 (20, 10)
1109
행: 20 열: 9 A214180 조회완료 (20, 10)
1110
행: 20 열: 9 A214270 조회완료 (20, 10)
1111
행: 20 열: 9 A214310 조회완료 (20, 10)
1112
행: 20 열: 9 A214370 조회완료 (20, 10)
1113
행: 20 열: 9 A214430 조회완료 (20, 10)
1114
행: 20 열: 9 A214450 조회완료 (20, 10)
1115
행: 20 열: 9 A214680 조회완료 (20, 10)
1116
행: 20 열: 9 A214870 조회완료 (20, 10)
1117
행: 20 열: 9 A215000 조회완료 (20, 10)
1118
행: 20 열: 9 A215090 조회완료 (20, 10)
1119
행: 20 열: 9 A215100 조회완료 (20, 10)
1120
행: 20 열: 9 A215200 조회완료 (20, 10)
1121
행: 20 열: 9 A215360 조회완료 (20, 10)
1122
행: 20 열: 9 A215380 조회완료 (20, 10)
1123
행: 20 열: 9 A215480 조회완료 (20, 10)
1124
행: 20 열: 9 A215600 조회완료 (20, 10)
1125
행: 20 열: 9 A215790 조회완료 (20, 10)
1126
행: 20 열: 9 A216050 조회완료 (20, 10)
1127
행: 20 열: 9 A217190 조회완료 (20, 10)
1128
행: 20 열

In [16]:
df= df[df.종목코드 != 'A065160']

In [19]:
print(df.날짜.value_counts())
print(df.날짜.min())
print(df.날짜.max())
df.head(20)

20190430    1314
20190424    1314
20190409    1314
20190416    1314
20190508    1314
20190507    1314
20190429    1314
20190412    1314
20190418    1314
20190417    1314
20190425    1314
20190419    1314
20190502    1314
20190423    1314
20190415    1314
20190426    1314
20190503    1314
20190411    1314
20190422    1314
20190410    1314
Name: 날짜, dtype: int64
20190409
20190508


,종목코드,날짜,시가,고가,저가,종가,거래량,거래대금,상장주식수,시가총액
0,A000250,20190508,46500,47050,45900,46200,139383.0,6.470000e+09,22105000.0,1.021288e+12
1,A000250,20190507,47000,47400,46550,46700,145152.0,6.805000e+09,22105000.0,1.032303e+12
2,A000250,20190503,46000,48250,45800,47900,225213.0,1.066900e+10,22105000.0,1.058829e+12
3,A000250,20190502,45600,46450,45500,45850,132230.0,6.095000e+09,22105000.0,1.013514e+12
4,A000250,20190430,46100,46750,45350,45750,127972.0,5.860000e+09,22105000.0,1.011303e+12
5,A000250,20190429,46050,46550,45500,46200,88172.0,4.066000e+09,22105000.0,1.021251e+12
6,A000250,20190426,45350,46150,44850,45700,284874.0,1.298600e+10,22105000.0,1.010198e+12
7,A000250,20190425,46150,46750,45500,45900,93689.0,4.319000e+09,22105000.0,1.014619e+12
8,A000250,20190424,47050,47100,45600,46000,117244.0,5.399000e+09,22105000.0,1.016830e+12
9,A000250,20190423,46500,46700,45550,46600,75808.0,3.512000e+09,22105000.0,1.030093e+12


In [20]:
시작일자= '20190409'
종료일자= '20190515'

query= """
        SELECT * FROM '코스닥_일별주가' WHERE 날짜 BETWEEN '%s' AND '%s'
        ORDER BY 날짜 ASC
       """ %(시작일자, 종료일자)


conn= sqlite3.connect('./data/StockData3.sqlite')
df_org= pd.read_sql_query(query, con=conn)
conn.close()
df_org.head()

,종목코드,날짜,시가,고가,저가,종가,거래량,거래대금,상장주식수,시가총액
0,A000250,20190409,49300,49300,47850,48850,154069.0,7.485000e+09,22105000.0,1.079829e+12
1,A000440,20190409,7450,7500,7300,7490,5961.0,4.400000e+07,6227000.0,4.664000e+10
2,A001000,20190409,2150,2420,2100,2230,1004217.0,2.284000e+09,24277000.0,5.413700e+10
3,A001540,20190409,14200,14250,13600,13800,91976.0,1.274000e+09,13042000.0,1.799790e+11
4,A001810,20190409,2855,2895,2835,2885,24331.0,7.000000e+07,22137000.0,6.386500e+10


In [21]:
df_org.tail()

,종목코드,날짜,시가,고가,저가,종가,거래량,거래대금,상장주식수,시가총액
7885,A310840,20190416,2085,2085,2080,2080,17863.0,37000000.0,3420000.0,7.113000e+09
7886,A310870,20190416,2090,2090,2050,2050,25.0,0.0,3420000.0,7.011000e+09
7887,A311270,20190416,2040,2040,2035,2035,4732.0,10000000.0,3600000.0,7.326000e+09
7888,A313750,20190416,2010,2015,2010,2015,263.0,1000000.0,4650000.0,9.369000e+09
7889,A317030,20190416,2015,2015,2010,2015,19084.0,38000000.0,5461000.0,1.100300e+10


In [22]:
# 중복되는 날짜 삭제
conn= sqlite3.connect('./data/StockData3.sqlite')
cursor = conn.cursor()
# query = "DELETE from t1 WHERE name > 'f';"
시작일자= '20190409'
종료일자= '20190515'
query= """
        DELETE FROM '코스닥_일별주가' WHERE 날짜 BETWEEN '%s' AND '%s'
       """ %(시작일자, 종료일자)
cursor.execute(query)
conn.commit()
conn.close()

In [23]:
with sqlite3.connect('./data/StockData3.sqlite') as conn:
    cursor = conn.cursor()
    query = "insert or replace into 코스닥_일별주가(종목코드, 날짜, 시가, 고가, 저가, 종가, 거래량, 거래대금, 상장주식수, 시가총액) values(?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    cursor.executemany(query, df.values.tolist())
    conn.commit()

In [24]:
with sqlite3.connect('./data/StockData3.sqlite') as conn:
    df_org= pd.read_sql("SELECT * From '코스닥_일별주가'", con=conn) # if_exists='append'

In [25]:
df_org.tail(20)

,종목코드,날짜,시가,고가,저가,종가,거래량,거래대금,상장주식수,시가총액
2503918,A317030,20190508,2040,2040,2025,2035,40928.0,83000000.0,5461000.0,1.111300e+10
2503919,A317030,20190507,2030,2040,2030,2035,6110.0,12000000.0,5461000.0,1.111300e+10
2503920,A317030,20190503,2025,2035,2020,2030,92171.0,187000000.0,5461000.0,1.108500e+10
2503921,A317030,20190502,2020,2025,2015,2020,122229.0,247000000.0,5461000.0,1.103100e+10
2503922,A317030,20190430,2020,2025,2020,2020,78874.0,159000000.0,5461000.0,1.103100e+10
2503923,A317030,20190429,2020,2025,2015,2020,56263.0,114000000.0,5461000.0,1.103100e+10
2503924,A317030,20190426,2015,2025,2015,2015,33475.0,68000000.0,5461000.0,1.100300e+10
2503925,A317030,20190425,2015,2020,2015,2020,17799.0,36000000.0,5461000.0,1.103100e+10
2503926,A317030,20190424,2025,2025,2015,2015,33827.0,68000000.0,5461000.0,1.100300e+10
2503927,A317030,20190423,2025,2025,2015,2025,29241.0,59000000.0,5461000.0,1.105800e+10


In [26]:
df_org.isnull().sum()

종목코드     0
날짜       0
시가       0
고가       0
저가       0
종가       0
거래량      0
거래대금     0
상장주식수    0
시가총액     0
dtype: int64